# Machine learning classifier for CMS publications
We use the CMS publications with the manual defined categories at:

https://cms-results.web.cern.ch/cms-results/public-results/publications/

to predict the category of a new CMS publication.

We will follow the `NaiveBayesClassifier` tutorial from `textblob` at:

https://textblob.readthedocs.io/en/dev/classifiers.html#classifiers

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from textblob.classifiers import NaiveBayesClassifier
nltk.download('punkt')  

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/restrepo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/restrepo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Load the data and prepare the training data set based in the title+abstract of each publication and the assigned category: `'CTG'`

In [2]:
df=pd.read_json('https://raw.githubusercontent.com/restrepo/cms_pub/main/cms.json')

Sample of the data

In [3]:
df[:2]

,CMS_id,report,title,journal,date,CTG,category,inspire_id,abstract
0,58,FSQ-16-006,Study of central exclusive $\pi^{+}\pi^{-}$ pr...,EPJC 80 (2020) 718,2020-03-05,FSQ,Forward and Small-x QCD Physics,1784063,Central exclusive and semiexclusive production...
1,57,FSQ-12-033,Measurement of single-diffractive dijet produc...,"EPJC 80, 1164 (2020)",2020-02-27,FSQ,Forward and Small-x QCD Physics,1782637,Measurements are presented of the single-diffr...


Prepare the data

In [4]:
df=df[df['abstract'].fillna('')!=''].reset_index(drop=True)
df['text']=df['title']+' '+df['abstract']
df=df.rename({'CTG':'label'},axis='columns')

stop_words=stopwords.words()
def simplify(text,stop_words=stop_words):
    text_tokens = text.replace("  "," ").split()
    filtered_words = [w for w in text_tokens if not w in stop_words]    
    return " ".join(filtered_words).lower()

df['text']=df['text'].apply(simplify)

print('total publication count:',df.shape)

total publication count: (912, 10)


shuffle the DataFrame

In [5]:
dfr=df.sample(df.shape[0])

Calculate the accuracy of the trained data set, `train`, with the `test` one

In [6]:
n_train=800
train=dfr[['text','label']][:n_train].reset_index(drop=True)
test =dfr[['text','label']][n_train:].reset_index(drop=True)

#Obtain the classifier with the `train` data set
cl = NaiveBayesClassifier(  
    [ (d.get('text'),d.get('label')) for d in train.to_dict(orient='records')]  )

# Check the accuracy of the classifier with the `test` dataset
print('accuracy → ',
      round( cl.accuracy( [ (d.get('text'),d.get('label')) for d in test.to_dict(orient='records')] ),2)
     )

accuracy →  0.85


Partial accuracies: there are several at the 90% level. Those would correspond to well defined categories

In [7]:
for c in test['label'].unique():
    print(c,'→',
          round(cl.accuracy( [ (d.get('text'),d.get('label')) for d in test[test['label']==c].to_dict(orient='records')]),2),
          f" samples → {test[test['label']==c].shape[0]}")

SUS → 1.0  samples → 11
EXO → 0.96  samples → 23
TOP → 0.82  samples → 17
HIN → 0.75  samples → 16
FSQ → 0.38  samples → 8
SMP → 1.0  samples → 13
HIG → 0.93  samples → 15
B2G → 0.4  samples → 5
BPH → 1.0  samples → 4


Check explicit results including the probability

In [8]:
fulltest=test.copy()
fulltest['test']=fulltest['text'].apply(cl.classify)

fulltest['prob']=fulltest['text'].apply(lambda t:   
                                cl.prob_classify( t ).prob(   cl.classify( t  )  ) ).round(2)

Show the ones that failed.
The following categories are expected to be mixed between them

* `[B2G,EXO,SUS]`
* `[HIG,TOP,SMP,FSQ]`

In this way, the errors are easy to understand. 

__Hypothesis__: The network analysis would have this failed publications in the frontiers of the clusters.

In [9]:
fulltest[fulltest['label']!=fulltest['test']]

,text,label,test,prob
3,${\upsilon\mathrm{(ns)}}$ polarizations versus...,HIN,BPH,1.00
11,observation top quark production proton-nucleu...,HIN,TOP,1.00
20,search two higgs bosons final states containin...,HIG,EXO,1.00
24,search heavy resonances decaying two higgs bos...,EXO,B2G,0.82
27,search $s$ channel single top quark production...,TOP,HIG,0.99
29,measurement four-jet production proton-proton ...,FSQ,SMP,1.00
39,measurement underlying event activity inclusiv...,FSQ,SMP,1.00
41,"study bose-einstein correlations pp, ppb, pbpb...",FSQ,HIN,1.00
51,measurement differential cross sections top qu...,TOP,SMP,0.83
55,coherent $\mathrm{ j } / \psi $ photoproductio...,HIN,SMP,0.81
